In [1]:
import open3d as o3d
import numpy as np
import torch
import trace1 as G
import numpy as np
from scipy.spatial import Delaunay

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [39]:
points = []
for i in np.linspace(0, 10, 100):
    for j in np.linspace(0, i/2, int(5*i)):
        noise = np.random.normal(scale=1e-2, size=(3))
        points.append([i,j,3+j] + noise)
        #points.append([i,j,3] + noise)

for i in np.linspace(10, 20, 100):
    for j in np.linspace(0, 5, int(5*i)):
        noise = np.random.normal(scale=1e-2, size=(3))
        points.append([i,j,3+j] + noise)
        #points.append([i,j,3] + noise)

for i in np.linspace(20, 30, 100):
    for j in np.linspace(i/2 - 10, 5, int(5*i)):
        noise = np.random.normal(scale=1e-2, size=(3))
        points.append([i,j,3+j] + noise)
        #points.append([i,j,3] + noise)

               
for k in range(10000):
    points.append(np.random.random(size=3)*30)

pcd= o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))

o3d.visualization.draw_geometries([pcd],
                                zoom=0.8412,
                                front=[0.4257, -0.2125, -0.8795],
                                lookat=[2.6172, 2.0475, 1.532],
                                up=[-0.0694, -0.9768, 0.2024])



In [40]:
pcd.estimate_normals()

In [41]:
planes = pcd.detect_planar_patches(normal_variance_threshold_deg=60, coplanarity_deg=75, outlier_ratio=0.75, min_plane_edge_length=0.0, min_num_points=0, search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)) #si potrebbe fare fine tuning
surfaces = []
edges = []
for i in range (8):
    for j in range (i+1,8):
        edges.append([i,j])
for plane in planes:
    vertices = np.asarray(plane.get_box_points())
    edges = sorted(edges, key= lambda edge: np.linalg.norm(vertices[edge[0]] - vertices[edge[1]]))
    #identifying the four shortest edges
    midpoints = [np.mean([vertices[edge[0]],vertices[edge[1]]], axis=0) for edge in edges[:4]]
    p0 = midpoints.pop(0)
    midpoints = sorted(midpoints, key= lambda point: np.linalg.norm(point - p0))
    p1 = midpoints[0]
    p2 = midpoints[1]
    points = [p0, p1, p2]
    #the surface is a section of the planar patch
    surfaces.append(G.Surface(np.array(points)))

In [42]:
len(surfaces)

1

In [43]:
import trace1 as G
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
G.plot_points(fig, np.array(pcd.points), size=0.5)
G.plot_surfaces(fig, surfaces)

In [44]:
steps_per_side = 4
cut_impurity = 0.2

planes = pcd.detect_planar_patches(normal_variance_threshold_deg=60, coplanarity_deg=75, outlier_ratio=0.75, min_plane_edge_length=0.0, min_num_points=0, search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)) #si potrebbe fare fine tuning
surfaces = []
edges = []
for i in range (8):
    for j in range (i+1,8):
        edges.append([i,j])
for plane in planes:
    print(plane)
    vertices = np.asarray(plane.get_box_points())
        
    target = len(plane.get_point_indices_within_bounding_box(pcd.points))
    edges = sorted(edges, key= lambda edge: np.linalg.norm(vertices[edge[0]] - vertices[edge[1]]))

    midpoints = [np.mean([vertices[edge[0]],vertices[edge[1]]], axis=0) for edge in edges[:4]]
    p0 = midpoints.pop(0)
    midpoints = sorted(midpoints, key= lambda point: np.linalg.norm(point - p0))
    p1 = midpoints[0]
    p2 = midpoints[1]
    points = [p0, p1, p2]
        #rectangle shape is the baseline
        #best_surface is [parallelogram (boolean), surface defining vertices (np.array (3,3)), area (float)]
    best_surface = [True, points, np.linalg.norm(p1 - p0) * np.linalg.norm(p2 - p0)]

    print("initial best surface: ", best_surface)


        #separating vertices in face up and face down
    short_edges = edges[:4]

    face_up_vertices = []
    face_down_vertices = []
    for short_edge in short_edges:
        v1 = vertices[short_edge[0]]
        v2 = vertices[short_edge[1]]

        if v1[2] != v2[2]:
            if v1[2] < v2[2]:
                face_down_vertices.append(v1)
                face_up_vertices.append(v2)
            else:
                face_down_vertices.append(v2)
                face_up_vertices.append(v1)

        else:
            if v1[1] != v2[1]:
                if v1[1] < v2[1]:
                    face_down_vertices.append(v1)
                    face_up_vertices.append(v2)
                else:
                    face_down_vertices.append(v2)
                    face_up_vertices.append(v1)

            else:
                if v1[0] != v2[0]:
                    if v1[0] < v2[0]:
                        face_down_vertices.append(v1)
                        face_up_vertices.append(v2)
                    else:
                        face_down_vertices.append(v2)
                        face_up_vertices.append(v1)

        
    face_up_vertices = sorted(face_up_vertices, key= lambda point: np.linalg.norm(point - face_down_vertices[0]))
    face_down_vertices = sorted(face_down_vertices, key= lambda point: np.linalg.norm(point - face_up_vertices[0]))
        #in both cases 1 is the closest vertex to 0, 2 is the second closest, 3 is the opposite one


    cuts_to_try = np.linspace(0, 1, steps_per_side)

        #trying to cut part of the rectangle to use a triangle
    print("base: ", np.linalg.norm(face_down_vertices[0] - face_down_vertices[2]), "altezza: ", np.linalg.norm(face_down_vertices[0] - face_down_vertices[1]))
    new_area = np.linalg.norm(face_down_vertices[0] - face_down_vertices[2]) * np.linalg.norm(face_down_vertices[0] - face_down_vertices[1]) * 0.5
    print("new_area triangles", new_area)
    best_impurity = cut_impurity
    if (new_area < best_surface[2]): 
        
        for cut in cuts_to_try:
                
                #noise introduction is necessary to build Delauney mesh (point must not be complanar)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[0] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[0] + (1-cut)*face_up_vertices[2], face_down_vertices[1], face_up_vertices[1]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[0] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[0] + (1-cut)*face_up_vertices[2], face_down_vertices[2], face_up_vertices[2]]) + noise)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target):    
                print("optimized with triangle type 1 with cut = ", cut)#######################
                best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                best_surface = [False, [np.mean([face_down_vertices[1], face_up_vertices[1]], axis=0),  np.mean([ cut*face_down_vertices[0] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[0] + (1-cut)*face_up_vertices[2]], axis=0), np.mean([face_down_vertices[3], face_up_vertices[3]], axis=0)], new_area]

            noise = np.random.normal(scale=1e-10, size=(6, 3))########################
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[3], face_down_vertices[1], face_up_vertices[1]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[3], face_down_vertices[2], face_up_vertices[2]]) + noise)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target):    
                    print("optimized with triangle type 2 with cut = ", cut)#######################
                    best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                    best_surface = [False, [np.mean([face_down_vertices[0], face_up_vertices[0]], axis=0),  np.mean([ cut*face_down_vertices[1] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[3]], axis=0), np.mean([face_down_vertices[2], face_up_vertices[2]], axis=0)], new_area]
                        
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[0],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[0], face_down_vertices[2], face_up_vertices[2]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[0],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[0], face_down_vertices[1], face_up_vertices[1]]) + noise)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target):    
                    print("optimized with triangle type 3 with cut = ", cut)#######################
                    best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                    best_surface = [False, [np.mean([face_down_vertices[2], face_up_vertices[2]], axis=0),  np.mean([cut*face_down_vertices[1] + (1-cut)*face_down_vertices[0],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[0]], axis=0), np.mean([face_down_vertices[3], face_up_vertices[3]], axis=0)], new_area]
                    break

            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[2] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[3], face_down_vertices[2], face_up_vertices[2]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[2] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[3], face_down_vertices[1], face_up_vertices[1]]) + noise)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target):    
                    print("optimized with triangle type 4 with cut = ", cut)#######################
                    best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                    best_surface = [False, [np.mean([face_down_vertices[0], face_up_vertices[0]], axis=0),  np.mean([cut*face_down_vertices[2] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[3]], axis=0), np.mean([face_down_vertices[1], face_up_vertices[1]], axis=0)], new_area]
                    break


        #trying to cut part of the rectangle to use a generic parallelogram
        

        #not trying disadvantageous cuts
    if best_surface[2] == new_area:
        cuts_to_try= cuts_to_try[:int(steps_per_side/2)]

    for cut in cuts_to_try:

            best_impurity = cut_impurity
            new_area = cut *  np.linalg.norm(face_down_vertices[0] - face_down_vertices[2]) * np.linalg.norm(face_down_vertices[0] - face_down_vertices[1])
            print("new_area parallelograms", new_area)

            parallelogram_optimization_found = False #flag to interrupt cycle if an acceptble parallelogram is found

            noise = np.random.normal(scale=1e-10, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[0] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[0] + (1-cut)*face_up_vertices[2], face_down_vertices[1], face_up_vertices[1]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[3] + (1-cut)*face_down_vertices[1],cut*face_up_vertices[3] + (1-cut)*face_up_vertices[1], face_down_vertices[2], face_up_vertices[2]]) + noise)
            print("impurity: ", sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) )/target, "best impurity: ", best_impurity)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target) and new_area < best_surface[2]:  
                print("optimized with parallelogram type 1 with cut = ", cut)#######################
                parallelogram_optimization_found = True
                best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                best_surface = [True, [np.mean([face_down_vertices[1], face_up_vertices[1]], axis=0),  np.mean([cut*face_down_vertices[0] + (1-cut)*face_down_vertices[2], cut*face_up_vertices[0] + (1-cut)*face_up_vertices[2]], axis=0), np.mean([cut*face_down_vertices[3] + (1-cut)*face_down_vertices[1],cut*face_up_vertices[3] + (1-cut)*face_up_vertices[1]], axis=0)], new_area]
            
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[2], face_up_vertices[2], cut*face_down_vertices[2] + (1-cut)*face_down_vertices[0],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[0], face_down_vertices[3], face_up_vertices[3]])) + noise)
            noise = np.random.normal(scale=1e-10, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[1], face_up_vertices[1], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[3], face_down_vertices[0], face_up_vertices[0]]) + noise)
            print("impurity: ", sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) )/target, "best impurity: ", best_impurity)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target) and new_area < best_surface[2]:
                print("optimized with parallelogram type 2 with cut = ", cut)#######################
                parallelogram_optimization_found = True
                best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                best_surface = [True, [np.mean([face_down_vertices[3], face_up_vertices[3]], axis=0),  np.mean([cut*face_down_vertices[2] + (1-cut)*face_down_vertices[0], cut*face_up_vertices[2] + (1-cut)*face_up_vertices[0]], axis=0), np.mean([cut*face_down_vertices[1] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[3]], axis=0)], new_area]            

            noise = np.random.normal(scale=1e-10, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[0], face_up_vertices[0], cut*face_down_vertices[0] + (1-cut)*face_down_vertices[1],cut*face_up_vertices[0] + (1-cut)*face_up_vertices[1], face_down_vertices[2], face_up_vertices[2]])) + noise)
            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[3], face_up_vertices[3], cut*face_down_vertices[3] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[3] + (1-cut)*face_up_vertices[2], face_down_vertices[1], face_up_vertices[1]]) + noise)
            print("impurity: ", sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) )/target, "best impurity: ", best_impurity)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target) and new_area < best_surface[2]:
                print("optimized with parallelogram type 3 with cut = ", cut)#######################
                parallelogram_optimization_found = True
                best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                best_surface = [True, [np.mean([face_down_vertices[1], face_up_vertices[1]], axis=0),  np.mean([cut*face_down_vertices[0] + (1-cut)*face_down_vertices[1], cut*face_up_vertices[0] + (1-cut)*face_up_vertices[1]], axis=0), np.mean([cut*face_down_vertices[3] + (1-cut)*face_down_vertices[2],cut*face_up_vertices[3] + (1-cut)*face_up_vertices[2]], axis=0)], new_area]

            noise = np.random.normal(scale=1e-12, size=(6, 3))
            hull_1 = Delaunay((np.array([face_down_vertices[1], face_up_vertices[1], cut*face_down_vertices[1] + (1-cut)*face_down_vertices[0],cut*face_up_vertices[1] + (1-cut)*face_up_vertices[0], face_down_vertices[3], face_up_vertices[3]])) + noise)
            noise = np.random.normal(scale=1e-10, size=(6, 3))
            hull_2 = Delaunay(([face_down_vertices[2], face_up_vertices[2], cut*face_down_vertices[2] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[3], face_down_vertices[0], face_up_vertices[0]]) + noise)
            print("impurity: ", sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) )/target, "best impurity: ", best_impurity)
            if (sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) < best_impurity * target) and new_area < best_surface[2]:
                print("optimized with parallelogram type 4 with cut = ", cut)#######################
                parallelogram_optimization_found = True
                best_impurity = sum(1 for point in pcd.points if (hull_1.find_simplex(point) >= 0 or hull_2.find_simplex(point) >= 0) ) / target
                best_surface = [True, [np.mean([face_down_vertices[3], face_up_vertices[3]], axis=0),  np.mean([cut*face_down_vertices[1] + (1-cut)*face_down_vertices[0], cut*face_up_vertices[1] + (1-cut)*face_up_vertices[0]], axis=0), np.mean([cut*face_down_vertices[2] + (1-cut)*face_down_vertices[3],cut*face_up_vertices[2] + (1-cut)*face_up_vertices[3]], axis=0)], new_area]

            if parallelogram_optimization_found:
                break


    surfaces.append(G.Surface(np.array(best_surface[1]), best_surface[0]))


OrientedBoundingBox: center: (14.7596, 2.49512, 5.49266), extent: 29.2599, 7.33666, 0.0484787)
initial best surface:  [True, [array([29.29509374,  5.34273933,  8.33903195]), array([29.47514409,  0.15815771,  3.15113213]), array([0.04402323, 4.83207319, 7.83418889])], 214.66987404179451]
base:  29.25988332400768 altezza:  7.336661997748236
new_area triangles 107.33493702089726
new_area parallelograms 0.0
impurity:  1.0 best impurity:  0.2
impurity:  0.9999522376653771 best impurity:  0.2
impurity:  1.0 best impurity:  0.2
impurity:  1.000047762334623 best impurity:  0.2
new_area parallelograms 71.55662468059816
impurity:  0.7529732053302766 best impurity:  0.2
impurity:  0.6171848879973253 best impurity:  0.2
impurity:  0.7825858527964847 best impurity:  0.2
impurity:  0.43396857238381814 best impurity:  0.2
new_area parallelograms 143.11324936119632
impurity:  0.4364999761188327 best impurity:  0.2
impurity:  0.1733295123465635 best impurity:  0.2
optimized with parallelogram type 2 wi

In [36]:
surfaces
surfaces[0].points

array([[ 2.13496516e-01, -4.87873320e-01,  2.99371118e+00],
       [ 9.82298858e+00,  4.99800125e+00,  2.99684841e+00],
       [ 1.96870534e+01, -1.57184573e-02,  2.99962189e+00]])

In [47]:
import trace1 as G
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
G.plot_points(fig, np.array(pcd.points), size=0.5)
G.plot_surfaces(fig, surfaces, color='#0000ff', opacity=1)